In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [17]:
from glob import glob
import os
import json


In [104]:
root_path = "../data/a9"
split = "test"

for split in ["train", "val", "test"]:
    img_label_s1_folder = os.path.join(
        root_path, split, "labels_images", "s110_camera_basler_south1_8mm"
    )
    img_label_s1_paths = sorted(glob(os.path.join(img_label_s1_folder, "*")))

    total_not_only_prev = 0
    total_not_only_next = 0
    total_not_both = 0

    data = {}

    for x in img_label_s1_paths:
        img_label_s1_json_path = os.path.join(x)
        json_data = None
        with open(img_label_s1_json_path, "r") as f:
            json_data = json.load(f)
            frame_idx = list(json_data["openlabel"]["frames"].keys())[0]
            frame_properties = json_data["openlabel"]["frames"][frame_idx]["frame_properties"]
            token = frame_properties["token"]
            prev = frame_properties["prev"]
            next = frame_properties["next"]
            if prev is None and next is None:
                total_not_both += 1
            elif prev is None:
                total_not_only_prev += 1
            elif next is None:
                total_not_only_next += 1

            data[token] = (prev, next, False)

    def rec_chain_check(x, only_left: bool = False, only_right: bool = False) -> int:
        prev = data[x][0]
        next = data[x][1]
        data[x] = (prev, next, True)
        size = 1
        if not only_right and data[x][0] is not None:
            size = rec_chain_check(data[x][0], True, False) + 1
        if not only_left and data[x][1] is not None:
            size = rec_chain_check(data[x][1], False, True) + 1
        return size

    chain_results = {}
    for k in data.keys():
        if not data[k][2]:
            res = rec_chain_check(k)
            if str(res) not in chain_results:
                chain_results[str(res)] = 1
            else:
                chain_results[str(res)] += 1

    print("\n", "-" * 15, "split: ", split, "-" * 15)

    chain_results_keys = [int(x) for x in chain_results.keys()]
    chain_results_keys.sort()
    chain_results = {str(i): chain_results[str(i)] for i in chain_results_keys}

    print("No. Sequences with following number of frames: ")
    for x, y in chain_results.items():
        if int(x) > 4:
            break
        print("\twith", x, "frames, total: \t", y)

    print("\nSamples with no prev: \t\t", total_not_only_prev)
    print("Samples with no next: \t\t", total_not_only_next)
    print("Samples with no prev nor next: \t", total_not_both)


 --------------- split:  train ---------------
No. Sequences with following number of frames: 

Samples with no prev: 		 4
Samples with no next: 		 4
Samples with no prev nor next: 	 0

 --------------- split:  val ---------------
No. Sequences with following number of frames: 
	with 1 frames, total: 	 24
	with 2 frames, total: 	 29
	with 3 frames, total: 	 9
	with 4 frames, total: 	 8

Samples with no prev: 		 61
Samples with no next: 		 61
Samples with no prev nor next: 	 24

 --------------- split:  test ---------------
No. Sequences with following number of frames: 
	with 1 frames, total: 	 35
	with 2 frames, total: 	 16
	with 3 frames, total: 	 5
	with 4 frames, total: 	 9

Samples with no prev: 		 35
Samples with no next: 		 35
Samples with no prev nor next: 	 35
